In [1]:
import numpy as np

#ポケモン一体の状態ベクトル化
def encode_pokemon(pokemon):
    #pokemonは辞書型で受け取る
    species_dict = {'ピカチュウ': 0, 'リザードン': 1, 'カビゴン': 2} #種族ID割り振り

    species_id = species_dict.get(pokemon['species'], -1)
    hp_ratio = pokemon['hp_ratio']
    status_map = {'なし': 0, 'まひ': 1, 'やけど': 2}
    status_id = status_map.get(pokemon['status'], 0)
    pp = np.array(pokemon['pp']) / 15  # PP最大15として正規化

    # ベクトル結合
    vector = np.array([species_id, hp_ratio, status_id])
    vector = np.concatenate([vector, pp])
    return vector

# テスト
poke_example = {'species': 'ピカチュウ', 'hp_ratio': 0.8, 'status': 'なし', 'pp': [5, 10, 10, 15]}
print(encode_pokemon(poke_example))

[0.         0.8        0.         0.33333333 0.66666667 0.66666667
 1.        ]


In [2]:
import numpy as np

def encode_pokemon(pokemon, species_dict, type_dict, ability_dict, item_dict):
    #種族(one-hot)
    species_vector = np.zeros(len(species_dict))
    if pokemon['species'] in species_dict:
        species_vector[species_dict[pokemon['species']]] = 1

    #HP
    hp_vector = np.array([pokemon['hp_ratio']]) #0.0~1.0

    #状態異常
    status_map = {'none': 0, 'par': 1, 'brn': 2, 'slp': 3, 'frz': 4, 'psn': 5, 'tox': 6}
    status_vector = np.zeros(len(status_map))
    if pokemon['status'] in status_map:
        status_vector[status_map[pokemon['status']]] = 1
    sleep_turns = np.array([pokemon.get('sleep_turns', 0) / 3])  # 最大3ターン仮定

    #技情報
    moves_vector = []
    for move in pokemon['moves']:
        type_id = type_dict.get(move['type'], 0)
        move_vec = [
            type_id / len(type_dict),       # タイプID正規化
            move['power'] / 250 if move['power'] else 0,  # 威力（最大250想定）
            move['priority'],               # 優先度（-7〜+5）
            move['pp'] / move['max_pp'],    # 残PP割合
            move['accuracy'] / 100 if move['accuracy'] else 1  # 命中率
        ]
        moves_vector.extend(move_vec)
    moves_vector = np.array(moves_vector)

    # 能力ランク（-6〜+6 → 0〜1に正規化） 
    stat_stages = np.array([(s + 6) / 12 for s in pokemon['stat_stages']])

    # 特性（one-hot）
    ability_vector = np.zeros(len(ability_dict))
    if pokemon['ability'] in ability_dict:
        ability_vector[ability_dict[pokemon['ability']]] = 1

    # 持ち物（one-hot)
    item_vector = np.zeros(len(item_dict))
    if pokemon['item'] in item_dict:
        item_vector[item_dict[pokemon['item']]] = 1

    # 結合
    return np.concatenate([
        species_vector,
        hp_vector,
        status_vector,
        sleep_turns,
        moves_vector,
        stat_stages,
        ability_vector,
        item_vector
    ])

In [1]:
import numpy as np

def encode_pokemon(pokemon, species_dict, type_dict, ability_dict, item_dict, moves_data, max_moves=4):
    """ポケモン情報をベクトル化する"""
    def one_hot(name, dictionary):
        vec = np.zeros(len(dictionary))
        if name in dictionary:
            vec[dictionary[name]] = 1
        return vec

    # 種族 (one-hot)
    species_vector = one_hot(pokemon['species'], species_dict)

    # HP割合
    hp_vector = np.array([pokemon.get('hp_ratio', 0.0)])

    # 状態異常 (one-hot)
    status_map = {'none': 0, 'par': 1, 'brn': 2, 'slp': 3, 'frz': 4, 'psn': 5, 'tox': 6}
    status_vector = np.zeros(len(status_map))
    if pokemon.get('status') in status_map:
        status_vector[status_map[pokemon['status']]] = 1
    sleep_turns = np.array([pokemon.get('sleep_turns', 0) / 3])

    # 技ベクトル（固定長 max_moves）
    moves_vector = []
    for i in range(max_moves):
        if i < len(pokemon['moves']):
            move_name = pokemon['moves'][i]
            if move_name in moves_data:
                move_info = moves_data[move_name]
                type_id = type_dict.get(move_info['type'], 0)
                move_vec = [
                    type_id / len(type_dict),  # タイプID正規化
                    (move_info.get('basePower') or 0) / 250,
                    move_info.get('priority', 0),
                    move_info.get('pp', 0) / max(move_info.get('maxpp', 1), 1),
                    (move_info.get('accuracy') if isinstance(move_info.get('accuracy'), (int, float)) else 100) / 100
                ]
            else:
                move_vec = [0, 0, 0, 0, 1]
        else:
            move_vec = [0, 0, 0, 0, 1]  # 技なし
        moves_vector.extend(move_vec)
    moves_vector = np.array(moves_vector)

    # 能力ランク
    stat_stages = np.array([(s + 6) / 12 for s in pokemon.get('stat_stages', [0]*6)])

    # 特性 & 持ち物 (one-hot)
    ability_vector = one_hot(pokemon.get('ability', ''), ability_dict)
    item_vector = one_hot(pokemon.get('item', ''), item_dict)

    return np.concatenate([
        species_vector,
        hp_vector,
        status_vector,
        sleep_turns,
        moves_vector,
        stat_stages,
        ability_vector,
        item_vector
    ])


# ===== テスト用データ =====
species_dict = {"Pikachu": 0, "Charizard": 1}
type_dict = {"Electric": 0, "Fire": 1, "Flying": 2}
ability_dict = {"Static": 0, "Blaze": 1}
item_dict = {"Light Ball": 0, "Leftovers": 1}
moves_data = {
    "Thunderbolt": {"type": "Electric", "basePower": 90, "priority": 0, "pp": 10, "maxpp": 16, "accuracy": 100},
    "Volt Switch": {"type": "Electric", "basePower": 70, "priority": 0, "pp": 20, "maxpp": 32, "accuracy": 100},
    "Protect": {"type": "Normal", "basePower": 0, "priority": 4, "pp": 10, "maxpp": 16, "accuracy": True},
    "Fake Out": {"type": "Normal", "basePower": 40, "priority": 3, "pp": 10, "maxpp": 16, "accuracy": 100},
}

pokemon_sample = {
    "species": "Pikachu",
    "hp_ratio": 0.75,
    "status": "par",
    "sleep_turns": 0,
    "moves": ["Thunderbolt", "Volt Switch", "Protect", "Fake Out"],
    "stat_stages": [0, 1, -1, 0, 0, 0],
    "ability": "Static",
    "item": "Light Ball"
}

# 実行テスト
encoded = encode_pokemon(pokemon_sample, species_dict, type_dict, ability_dict, item_dict, moves_data)
print("エンコード結果:", encoded)
print("ベクトル長:", len(encoded))


エンコード結果: [1.         0.         0.75       0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.36       0.         0.625      1.         0.         0.28
 0.         0.625      1.         0.         0.         4.
 0.625      0.01       0.         0.16       3.         0.625
 1.         0.5        0.58333333 0.41666667 0.5        0.5
 0.5        1.         0.         1.         0.        ]
ベクトル長: 41


In [3]:
// encode_pokemon.js
const fs = require('fs');
const path = require('path');

// 辞書を読み込み
const dicts = JSON.parse(fs.readFileSync(path.join(__dirname, 'data', 'dicts.json'), 'utf8'));

/**
 * ポケモンデータをエンコードしてID配列に変換
 * @param {Object} pokemon - { species, types, ability, item, moves }
 * @returns {Object} - { speciesId, typeIds, abilityId, itemId, moveIds }
 */
function encodePokemon(pokemon) {
  return {
    speciesId: dicts.species_dict[pokemon.species] ?? null,
    typeIds: pokemon.types.map(type => dicts.type_dict[type] ?? null),
    abilityId: dicts.ability_dict[pokemon.ability] ?? null,
    itemId: dicts.item_dict[pokemon.item] ?? null,
    moveIds: pokemon.moves.map(move => dicts.move_dict[move] ?? null)
  };
}

// --- 動作テスト ---
const examplePokemon = {
  species: "Pikachu",
  types: ["Electric"],
  ability: "Static",
  item: "Light Ball",
  moves: ["Thunderbolt", "Volt Tackle", "Protect", "Fake Out"]
};

console.log(encodePokemon(examplePokemon));


SyntaxError: invalid syntax (4270982857.py, line 1)